<a href="https://colab.research.google.com/github/TimeryaevaM/new_rep/blob/master/diplom_2v.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [399]:
import pandas as pd
import os
import csv
import numpy as np

In [400]:
# df = pd.read_csv("/data_new.csv", sep=",", quoting=csv.QUOTE_ALL, encoding='utf-8')

df = pd.read_csv("/data_new.csv")

In [401]:
df.head()

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
0,Active,NaN,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,NaN,NaN,611019,"$418,000"
1,for sale,NaN,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,NaN,201916904,"$310,000"
2,for sale,NaN,single-family home,2005 Westridge Rd,2 Baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,1.0,NaN,yes,FR19221027,"$2,895,000"
3,for sale,NaN,single-family home,4311 Livingston Ave,8 Baths,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",yes,Dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",75205,5 Beds,TX,3.0,NaN,NaN,14191809,"$2,395,000"
4,for sale,NaN,lot/land,1524 Kiscoe St,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,NaN,NaN,NaN,861745,"$5,000"


In [402]:
df.shape

(377185, 18)

In [403]:
df[df['target'].isna()].shape

(2481, 18)

In [404]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377185 entries, 0 to 377184
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        337267 non-null  object
 1   private pool  4181 non-null    object
 2   propertyType  342452 non-null  object
 3   street        377183 non-null  object
 4   baths         270847 non-null  object
 5   homeFacts     377185 non-null  object
 6   fireplace     103115 non-null  object
 7   city          377151 non-null  object
 8   schools       377185 non-null  object
 9   sqft          336608 non-null  object
 10  zipcode       377185 non-null  object
 11  beds          285903 non-null  object
 12  state         377185 non-null  object
 13  stories       226470 non-null  object
 14  mls-id        24942 non-null   object
 15  PrivatePool   40311 non-null   object
 16  MlsId         310305 non-null  object
 17  target        374704 non-null  object
dtypes: object(18)
memory usa

In [405]:
print("Percent NULL values\n")
round(df.isna().sum() / df.shape[0] * 100, 2)

Percent NULL values



status          10.58
private pool    98.89
propertyType     9.21
street           0.00
baths           28.19
homeFacts        0.00
fireplace       72.66
city             0.01
schools          0.00
sqft            10.76
zipcode          0.00
beds            24.20
state            0.00
stories         39.96
mls-id          93.39
PrivatePool     89.31
MlsId           17.73
target           0.66
dtype: float64

Удаляем столбцы, где процент NULL значений больше 40%

In [406]:
df.drop(['private pool', 'fireplace', 'mls-id', 'PrivatePool'], axis=1, inplace=True)

In [407]:
df.nunique()

status             159
propertyType      1280
street          337076
baths              229
homeFacts       321009
city              2026
schools         297365
sqft             25405
zipcode           4549
beds              1184
state               39
stories            348
MlsId           232944
target           43939
dtype: int64

status

In [408]:
df['status'] = df['status'].str.lower()

In [409]:
import string
string.punctuation
string.digits

'0123456789'

In [410]:
df['status'] = df['status'].str.replace('[{}]'.format(string.punctuation), ' ').str.replace('[{}]'.format(string.digits), ' ').apply(lambda x: " ".join(str(x).split()))
df['status'].value_counts()

for sale                   199571
active                     105207
nan                         39918
foreclosure                  6769
new construction             5475
                            ...  
contingent sale of home         1
temporary active                1
reactivated                     1
pending take backups            1
conditional contract            1
Name: status, Length: 104, dtype: int64

In [411]:
df['status'].nunique()

104

propertyType 

In [412]:
df['propertyType'] = df['propertyType'].str.lower()
df['propertyType'] = df['propertyType'].str.replace('[{}]'.format(string.punctuation), ' ').apply(lambda x: " ".join(str(x).split()))
df['propertyType'] = df['propertyType'].str.replace('one', '1').str.replace('two', '2')

In [413]:
df['propertyType'].value_counts()[20:40]

contemporary modern           1010
apartment                      922
2 stories                      911
mfd mobile home                792
single detached                751
detached 2 story               638
2 story                        596
transitional                   560
cooperative                    535
florida                        530
bungalow                       442
farms ranches                  371
garden home                    305
cape cod                       281
2 stories traditional          280
spanish mediterranean          272
other style                    249
1 story traditional            238
other                          232
condo townhome traditional     225
Name: propertyType, dtype: int64

In [414]:
df['stories_dop'] = df['propertyType'].str.findall(r'[a-z0-9]+ story|[a-z0-9]+ stories').apply(lambda x: None if len(x) == 0 else x[-1])
#.str.replace('one', '1').str.replace('two', '2')
df['stories_dop'].value_counts(dropna=False)

NaN          369527
1 story        4440
2 stories      1558
2 story        1466
3 stories       134
8 stories        38
7 stories        15
3 story           6
5 story           1
Name: stories_dop, dtype: int64

In [415]:
set_story = set(df['stories_dop'].dropna())
for y in set_story:
  df['propertyType'] = df['propertyType'].str.replace(y, '')

In [416]:
df[df['propertyType'].str.contains('home')]['propertyType'].value_counts()

single family home                                                                        123934
condo townhome row home co op                                                               7701
multi family home                                                                           1501
mfd mobile home                                                                              792
garden home                                                                                  305
                                                                                           ...  
condo townhome lake house vacation home traditional                                            1
condo townhome contemporary modern mediterranean spanish                                       1
garden zero lot line single detached vacation home mediterranean traditional                   1
condo townhome ranch                                                                           1
condo townhome hi rise histori

In [417]:
df['propertyType'] = df['propertyType'].str.replace('home', '')
df['propertyType'] = df['propertyType'].apply(lambda x: None if x == '' else  " ".join(str(x).split()))

In [418]:
df['propertyType'].nunique()

1113

street

In [419]:
df['street'].value_counts(dropna=False)[1200:]

8502 Jubilee Ct           3
1614 34th St NW           3
3222 Alabama Ct           3
1909 W 74th St            3
11142 Knight Castle Dr    3
                         ..
94 Waltham St #2          1
311 Road 5033             1
16310 Tamiami Trl         1
6520 SW 76th St           1
5339 Rushmore Dr          1
Name: street, Length: 335877, dtype: int64

In [420]:
df['street'] = df['street'].str.lower()
df['street'] = df['street'].apply(lambda x: None if 'not disclosed' in str(x) else 
                                  None if 'undisclosed' in str(x) else
                                  None if 'not available' in str(x) else
                                  None if 'unknown' in str(x) else str(x))

In [421]:
df.loc[(df['street'].isna()==False) & (df['street'].str.contains(r'mls ')), 'street'] = None

In [422]:
df.loc[81361, 'street']

'1131 w 24th street st unit c '

In [423]:
punct = string.punctuation
df['street'] = df['street'].str.replace('[{}]'.format(string.punctuation), ' ').str.replace(r'\b[0-9]+\b', '').str.replace('  ', ' ').str.strip()

In [424]:
street_list_tuple = [('st', 'street'), ('dr', 'drive'), ('ave', 'avenue'), ('rd', 'road'), ('rdg', 'ridge'), ('blvd', 'boulevard'), ('pkwy', 'parkway'), 
                   ('cir', 'circle'), ('hwy', 'highway'), ('ln', 'lane'), ('lndg', 'landing'), ('trl', 'trail'), ('ter', 'terrace'), ('ct', 'court'), 
                   ('crk', 'creek'), ('pl', 'place'), ('rds', 'roads'), ('bnd', 'bend'), ('fls', 'falls'), ('clf', 'cliff'), ('clfs', 'cliff'), ('mtn', 'mountain'), 
                   ('crst', 'crest'), ('hl', 'hill'), ('ml', 'mill'), ('mnr', 'manor'), ('hts', 'heights')]       
                          
street_list_2 = ['street', 'drive', 'avenue', 'road', 'ridge', 'way', 'boulevard', 'parkway', 'circle', 'highway', 'lane', 
                 'landing', 'trail', 'terrace', 'court', 'creek', 'place', 'roads', 'bend', 'falls', 'cliff', 'mountain', 
                 'crest', 'hill', 'mill', 'point', 'loop', 'court', 'manor', 'heights', 'oaks', 'park']

In [425]:
def street_replace(df, st_tupl):

  # Функция замены коротких синонимов на полные названия и удаление дубликатов

  df['street'] = df['street'].str.replace(rf'\b{st_tupl[0]}\b', st_tupl[1]).str.replace(rf'\b{st_tupl[1]}\b \b{st_tupl[1]}\b', st_tupl[1])
  df['street'] = df['street'].str.replace('  ', ' ').str.strip()

  # print(f'replace {st_tupl[0]} to {st_tupl[1]}')

In [426]:
for x in street_list_tuple:
  street_replace(df, x)

In [427]:
df['street'].sample(20)

6370               lorient terrace
370460       carrollsburg place sw
156485              ne 21st avenue
63647              summerset drive
328116                 lahser road
24453            v l edgerton road
175299             swallow lane 5d
72879           collins avenue apt
350587               heyler street
37712             nw 194th terrace
297687         thicket bluff drive
159005            taylor street nw
119147      e monument avenue unit
48613     ne 144th street apt a101
285572               walker street
241869          santa anita avenue
280837              s 243rd street
189178            manhattan avenue
231542       buckskin jumper drive
207082                   monaltrie
Name: street, dtype: object

In [428]:
def del_dropwords_street(df, dropword):

  # Удаляем dropword из признака street
  df['street'] = df['street'].str.replace(rf'\b{dropword}\b', '').str.replace(r'  ', ' ').str.strip()

  print(f"Drop {dropword}")

In [429]:
dropwords = ['lot', 'lots', 'collections', 'collection', 'plan', 'apt', 'unit', 'buildable', 'the']

for x in dropwords:
  del_dropwords_street(df, x)

Drop lot
Drop lots
Drop collections
Drop collection
Drop plan
Drop apt
Drop unit
Drop buildable
Drop the


In [430]:
# Удаляем мусор из строк, чтобы легче паттерны было выделять
df['street'] = df['street'].str.replace(r'\b[a-z]{1,2}\b', '').str.replace('  ', ' ').str.strip()
df['street'] = df['street'].str.replace(r'\b[0-9]+[a-z]\b', '').str.replace(r'\b[a-z]+[0-9]\b', '').str.replace(r'\b[0-9]+ft\b', '')
df['street'] = df['street'].str.replace(r'  ', ' ').str.strip()

In [431]:
def street_var(df, street_var):
  #### Создаем новые столбцы для значений параметров улиц

  # print(street_var + " :")
  street_col = street_var + '_v'
  df[street_col] = None

  df[street_col] = df['street'].apply(lambda x: None if x is None else 1 if x.endswith(' ' + street_var) else 
                                      1 if ' ' + street_var + ' ' in x else 0)

  # print(f'New columm {street_col} is ready')

In [432]:
for x in street_list_2:
  street_var(df, x)

In [433]:
df['street'] = df['street'].apply(lambda x: None if x is None else
                                  None if x.strip() == '' else x)

In [434]:
def change_street(df, spec_str):

  ### Преобразуем улицы согласно паттернам

  new_street = df.loc[(df['street'].isna()==False) & (df['street'].str.contains(r'\b[a-z]{3,}\b \b[a-z]{2,}\b ' + spec_str)), 'street']
  new_street = new_street.str.findall(r'\b[a-z]{3,}\b \b[a-z]{2,}\b ' + spec_str).apply(lambda x: None if len(x) == 0 else x[0])
  df.loc[new_street.index, 'street'] = new_street

  new_street = df.loc[(df['street'].isna()==False) & (df['street'].str.contains('[0-9]+[a-z]+ ' + spec_str)), 'street']
  new_street = new_street.str.findall('[0-9]+[a-z]+ ' + spec_str).apply(lambda x: None if len(x) == 0 else x[0])
  df.loc[new_street.index, 'street'] = new_street

  new_street = df.loc[(df['street'].isna()==False) & (df['street'].str.contains('[0-9]+[a-z]+ [a-z]+ ' + spec_str)), 'street']
  new_street = new_street.str.findall('[0-9]+[a-z]+ [a-z]+ ' + spec_str).apply(lambda x: None if len(x) == 0 else x[0])
  df.loc[new_street.index, 'street'] = new_street

  # new_street = df.loc[(df['street'].isna()==False) & (df['street'].str.contains(r'\b[a-z]{3,}-[a-z]{3,}\b '+ spec_str)), 'street']
  # new_street = new_street.str.findall(r'\b[a-z]{3,}-[a-z]{3,}\b '+ spec_str).apply(lambda x: None if len(x) == 0 else x[0])
  # df.loc[new_street.index, 'street'] = new_street

  new_street = df.loc[(df['street'].isna()==False) & (df['street'].str.contains(r'^\b[a-z]{3,}\b '+ spec_str)), 'street']
  new_street = new_street.str.findall(r'^\b[a-z]{3,}\b '+ spec_str).apply(lambda x: None if len(x) == 0 else x[0])
  df.loc[new_street.index, 'street'] = new_street
  
  # print('Change', spec_str)

In [435]:
for x in street_list_2:
  change_street(df, x)

In [436]:
# Создаем столбец длины улиц, чтобы ими управлять если что

df['street_length'] = df['street'].apply(lambda x: 0 if x is None else len(x.split()) )

In [437]:
def del_var_from_street(df, street_var):

  # Функция удаления переменных улиц для уменьшения количества уникальных значений

  df['street'] = df['street'].str.replace(rf'\b{street_var}\b', '').str.replace('  ', ' ').str.strip()
  df['street'] = df['street'].apply(lambda x: None if x == ''  else x)

  # print(f'Del {street_var} from street')

In [438]:
for x in street_list_2:
  del_var_from_street(df, x)

In [439]:
df[df['street_length'] == 2]['street'].nunique()
# Нужно использовать токенайзер

34070

In [440]:
# Слишком много уникальных значений даже с учетом паттернов
# Уменьшим их количество по объему

In [441]:
# df['street'].value_counts()[df['street'].value_counts() >= 50]

In [442]:
# Проверить данный способ, если нет, использовать tokenizer

In [443]:
df.sample(5)

,status,propertyType,street,baths,homeFacts,city,schools,sqft,zipcode,beds,state,stories,MlsId,target,stories_dop,street_v,drive_v,avenue_v,road_v,ridge_v,way_v,boulevard_v,parkway_v,circle_v,highway_v,lane_v,landing_v,trail_v,terrace_v,court_v,creek_v,place_v,roads_v,bend_v,falls_v,cliff_v,mountain_v,crest_v,hill_v,mill_v,point_v,loop_v,manor_v,heights_v,oaks_v,park_v,street_length
84097,for sale,single family,temple,2 Baths,"{'atAGlanceFacts': [{'factValue': '1927', 'fac...",College Park,"[{'rating': ['1/10', '3/10', '2/10'], 'data': ...","1,611 sqft",30337,3 Beds,GA,1.0,6658976,"$344,900",None,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
363747,for sale,single family,briargrove,3 Baths,"{'atAGlanceFacts': [{'factValue': '1993', 'fac...",Dallas,"[{'rating': ['4/10', '4/10', '6/10'], 'data': ...","2,490 sqft",75287,4 Beds,TX,2.0,14166153,"$318,000",None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
87879,under contract show,single family,glenham,2,"{'atAGlanceFacts': [{'factValue': '1959', 'fac...",Charlotte,"[{'rating': ['4', '2', '7', '5', '8', '1', 'NR...",1504,28210,3,NC,1 Story,3552329,"$423,500",None,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
327344,active,single detached traditional,sherbert,NaN,"{'atAGlanceFacts': [{'factValue': '2007', 'fac...",North Richland Hills,"[{'rating': ['7', '10', '7'], 'data': {'Distan...","3,105",76180,NaN,TX,NaN,14223214,"425,000",None,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
337219,active,single family,halleck,2,"{'atAGlanceFacts': [{'factValue': '1909', 'fac...",Seattle,"[{'rating': ['9', '7', '5', '6', '6', 'NR', 'N...",1530,98116,2,WA,NaN,1538950,"$679,000",None,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2


baths

In [444]:
df['baths'] = df['baths'].apply(lambda x: None if x is np.nan else x)
df['baths'].value_counts(dropna=False)

NaN           106338
2 Baths        52466
3 Baths        35506
2              20452
2.0            16576
               ...  
3.3 Baths          1
5.25 Baths         1
29                 1
1-2 Baths          1
0.5 Baths          1
Name: baths, Length: 230, dtype: int64

In [445]:
punct = '~—\-:'

In [446]:
df['baths'] = df['baths'].str.lower().str.replace('[a-z]+\.?', '').str.replace(',', '.').str.replace('[{}]'.format(punct), ' ')
df['baths'] = df['baths'].apply(lambda x: None if str(x).strip() =='' else x)

In [447]:
df['baths_new'] = df['baths'].str.findall('[0-9]+\.?[0-9]*')
df['baths_new'] = df['baths_new'].apply(lambda x: None if x is None else x[0]).astype(float)
df['baths_new'].value_counts()

# df['baths_new'].to_csv('out.csv')
# ванна 0.5 проверить

2.0     103519
3.0      66683
4.0      26333
1.0      17787
2.5      13786
         ...  
1.8          1
64.0         1
14.5         1
0.5          1
42.0         1
Name: baths_new, Length: 84, dtype: int64

city

In [448]:
df['city'] = df['city'].str.lower().str.strip()
df['city'] = df['city'].str.replace('\-', ' ')

In [449]:
df['city'] = df['city'].str.replace(' city', '').str.replace(' town+[a-z]*', '')
df['city'] = df['city'].str.replace('st\.', 'saint').str.replace(' twp.', '').str.replace('e. ', '').str.replace('ft.', 'fort').str.replace('s.a.', 'san antonio')
df['city'] = df['city'].str.replace(' twp', '').str.replace(' vlg', ' village')
df['city'] = df['city'].str.replace(' hts', 'heights').str.replace(' pt', 'point').str.replace('pt ', 'port ')
df['city'] = df['city'].str.replace('^st ', 'saint ')

In [450]:
df.loc[(df['city'].isna()==False) & (df['city'].str.contains('san antonio')), 'city'] = 'san antonio'

In [451]:
df['city'] = df['city'].str.replace('p c ', 'panama ').str.replace('^n ', 'north ').str.replace("town 'n' country", 'town north country')
df['city'] = df['city'].str.replace('nyork', 'new york').str.replace('nyc', 'new york')
df['city'] = df['city'].str.replace('bed-stuy', 'brooklyn').str.replace('collegepoint', 'college point')
df['city'] = df['city'].str.replace('othvalue [ a-z]*', 'othvalue')

In [452]:
df['city'] = df['city'].apply(lambda x: " ".join(str(x).split()))

In [453]:
df['city'].value_counts()

san antonio               30033
houston                   24442
miami                     15528
jacksonville              10032
dallas                     8858
                          ...  
hollywood hills               1
ave                           1
willow spring                 1
outside ar(outside ca)        1
cherry valley                 1
Name: city, Length: 1785, dtype: int64

sqft

In [454]:
df['sqft'].value_counts()

0              11854
1,200 sqft       839
1,000 sqft       654
1,100 sqft       573
1,800 sqft       563
               ...  
85,062 sqft        1
6,086              1
6680               1
5,094 sqft         1
12,380             1
Name: sqft, Length: 25405, dtype: int64

In [455]:
df['sqft'] = df['sqft'].apply(lambda x: None if x is np.nan else x)

In [456]:
df['sqft'] = df['sqft'].str.replace('[a-z ]*: ', '')
df['sqft'] = df['sqft'].str.replace('-','').str.replace('[A-Za-z]+','').str.replace(',','')

In [457]:
df['sqft'] = df['sqft'].str.strip().apply(lambda x: None if x is None else 
                                          None if x == '' else int(x))

In [458]:
df['sqft'].value_counts()

0.0         11854
1200.0       1426
1000.0       1027
1500.0       1014
1800.0        988
            ...  
8966.0          1
11089.0         1
123275.0        1
44351.0         1
32767.0         1
Name: sqft, Length: 9897, dtype: int64

zipcode

In [459]:
df['zipcode'].value_counts()

32137         2141
33131         1563
34747         1488
78245         1390
34759         1333
              ... 
00000            1
27614-8755       1
10129            1
63117            1
41754            1
Name: zipcode, Length: 4549, dtype: int64

In [460]:
df['zipcode'] = df['zipcode'].str.replace('\-[0-9]*', '').apply(lambda x: None if x is None else None if x == '' else int(x))

beds

In [461]:
df['beds'] = df['beds'].apply(lambda x: None if x is np.nan else x).str.lower()

In [462]:
df['beds'].value_counts(dropna=False)

NaN                                                                                                                           91282
3 beds                                                                                                                        53459
4 beds                                                                                                                        35418
3                                                                                                                             31406
2 beds                                                                                                                        26362
                                                                                                                              ...  
based on redfin's st johns data, we estimate the home's value is $360,731, which is 2.2% less than its current list price.        1
8.2 acres                                                                   

In [463]:
index_sqft = df.loc[(df['sqft'].isna()) & (df['beds'].str.contains('sqft')), ['sqft','beds']].index
new_sqft = df.loc[(df['sqft'].isna()) & (df['beds'].str.contains('sqft')), ['sqft','beds']]
new_sqft['sqft'] = new_sqft['beds'].str.strip().str.replace(',', '.').str.findall('[0-9]+\.*[0-9]*').apply(lambda x: None if len(x) == 0 else x[0]).astype(float)

df.loc[(df['sqft'].isna()) & (df['beds'].str.contains('sqft')), 'sqft'] = new_sqft['sqft']
df.loc[(df['beds'].isna()==False) & (df['beds'].str.contains('sqft')), 'beds'] = None

In [464]:
index_sqft = df.loc[(df['sqft'].isna()) & (df['beds'].str.contains('acre')), ['sqft','beds']].index
new_sqft = df.loc[(df['sqft'].isna()) & (df['beds'].str.contains('acre')), ['sqft','beds']]
new_sqft['sqft'] = new_sqft['beds'].str.strip().str.replace(',', '.').str.findall('[0-9]+\.*[0-9]*').apply(lambda x: None if len(x) == 0 else x[0]).astype(float)
new_sqft['sqft'] = (new_sqft['sqft'] * 43560).astype(int)

df.loc[(df['sqft'].isna()) & (df['beds'].str.contains('acre')), 'sqft'] = new_sqft['sqft']
df.loc[(df['beds'].isna()==False) & (df['beds'].str.contains('acre')), 'beds'] = None

In [465]:
df['beds'] = df['beds'].str.replace('bedrooms', 'beds').str.replace('bd', 'beds')

In [466]:
df.loc[(df['beds'].isna()==False) & (df['beds'].str.strip().str.contains('^[a-z,\-%@#_:;~ ]+$')), 'beds'] = None

In [467]:
df['beds'].value_counts(dropna=False)[:20]

NaN       111390
3 beds     66336
4 beds     43683
2 beds     31605
3          31406
4          20030
2          16110
5 beds     13904
3.0         8088
5           6424
4.0         5231
1           4610
6 beds      4466
2.0         2430
6           1806
5.0         1372
7 beds      1247
0           1230
1 beds      1081
8 beds       856
Name: beds, dtype: int64

In [468]:
new_bath = df.loc[(df['baths_new'].isna()) & (df['beds'].str.contains('bath')), 'beds'].str.findall('[0-9a-z ]+ bath').apply(lambda x: x[0].split()[0]).astype(float)
df.loc[new_bath.index, 'baths_new'] = new_bath
df.loc[new_bath.index, 'baths_new']

18787     1.0
124819    1.0
172605    1.0
238222    1.0
279149    2.0
325249    1.0
Name: baths_new, dtype: float64

In [469]:
new_beds = pd.concat([(df.loc[(df['beds'].isna()==False) & (df['beds'].str.strip().str.contains('[0-9 a-z]+bed[a-z]*,')), 'beds']),
                      df.loc[(df['beds'].isna()==False) & (df['beds'].str.strip().str.contains(',[0-9 a-z]+bed[a-z]*')), 'beds']
                      ],
                     ).reset_index()
new_beds = new_beds.drop_duplicates().set_index('index')
new_beds['beds'] = new_beds['beds'].str.replace('or more ', '').str.findall('[0-9 ]+bed').apply(lambda x: x[0].split()[0]).astype(float)

df.loc[new_beds.index, 'beds'] = new_beds['beds']
df.loc[new_beds.index, 'beds']

index
18787     2
172605    2
200331    3
238222    3
325249    2
124819    2
279149    3
Name: beds, dtype: object

In [470]:
punct = '!@#_,:;&'
df.loc[(df['beds'].isna()==False) & (df['beds'].str.contains('[{}]'.format(punct))), 'beds'] = None
punct = '\-'
new_beds = df.loc[(df['beds'].isna()==False) & (df['beds'].str.contains('[{}]'.format(punct))), 'beds'].str.findall('[0-9]+').apply(lambda x: x[-1]).astype(float)
df.loc[new_beds.index, 'beds'] = new_beds
df.loc[new_beds.index, 'beds']

330468    2
Name: beds, dtype: object

In [471]:
df['beds'] = df['beds'].str.replace('or more ', '')
df['beds'] = df['beds'].str.replace('beds','').str.strip().apply(lambda x: None if x is None else
                                                                 None if x == '' else x).astype(float)

state

In [472]:
df['state'].value_counts(dropna=False)[-20:]

UT    1325
MD    1090
VT     868
MO     866
VA     801
WI     452
NJ     436
ME     259
IA     242
KY      90
OK      49
MS      40
SC      28
MT       7
DE       5
BA       1
Fl       1
AL       1
OT       1
OS       1
Name: state, dtype: int64

stories

In [473]:
df['stories_dop'] = df['stories_dop'].apply(lambda x: None if x is None else x.split()[0]).astype(float)
df['stories_dop'].value_counts(dropna=False)

NaN    369527
1.0      4440
2.0      3024
3.0       140
8.0        38
7.0        15
5.0         1
Name: stories_dop, dtype: int64

In [474]:
df['stories'] = df['stories'].str.lower()
df['stories'].value_counts(dropna=False)

NaN                     150715
1.0                      67454
2.0                      55283
1                        23086
2                        18146
                         ...  
tudor                        1
2 stories, townhouse         1
95.0                         1
condo 5+ stories             1
dwelling stories             1
Name: stories, Length: 349, dtype: int64

In [475]:
punct = '!|/_\-:;'
df.loc[(df['stories'].isna()==False) & (df['stories'].str.contains('[{}]'.format(punct))), 'stories'].value_counts()
df.loc[(df['stories'].isna()==False) & (df['stories'].str.contains('[0-9 ]/[0-9 ]')), 'stories'].value_counts()
df.loc[(df['stories'].isna()==False) & (df['stories'].str.contains('half')), 'stories'].value_counts()
df.loc[(df['stories'].isna()==False) & (df['stories'].str.contains('multi')), 'stories'].value_counts()

# много пустых значений, не знаю, обрабатывать ли их

multi/split                   271
multi-level                    57
other multi-unit               19
three or more, multi/split      7
multi level                     6
multi-story                     3
two, multi/split                3
one, two, multi/split           1
other, two, multi/split         1
1 story, multi level            1
multi-level, new englander      1
multi level, three or more      1
multi level, tri level          1
3 story, multi level            1
tri/multi level                 1
multi/split, two                1
2 story, multi level            1
Name: stories, dtype: int64

MlsId

In [476]:
df = df.drop('MlsId', axis=1)

homeFacts

In [477]:
df['homeFacts'] = df['homeFacts'].apply(lambda x: eval(x))

In [499]:
df['homeFacts_new'] = df['homeFacts'].apply(lambda x: list(x.values())[0])

In [529]:
list_columns = ['Year built', 'Remodeled year', 'Heating', 'Cooling', 'Parking', 'lotsize', 'Price/sqft']

for column in enumerate(list_columns):
  df[column[1]] = None
  df[column[1]] = df['homeFacts_new'].apply(lambda x: None if x[column[0]].get('factValue') == '' else  x[column[0]].get('factValue'))

In [532]:
# Выбрать признаки из lotsize, Price/sqft для тех значений, которые пустые в соответствующих столбцах
# Преобразовать те признаки, в которых мало null значений

schools

target

In [346]:
df['target'].value_counts(dropna=False)

NaN           2481
$225,000      1462
$275,000      1355
$250,000      1312
$350,000      1296
              ... 
9,045,000        1
$115,077         1
$64,923          1
$363,406         1
$2,210,812       1
Name: target, Length: 43940, dtype: int64

In [347]:
df['target'] = df['target'].apply(lambda x: None if x is np.nan else x)

In [348]:
df[(df['target'].isna()==False) & (df['target'].str.isdigit())]['target'].value_counts()

800    2
3      2
1      2
825    1
25     1
20     1
8      1
30     1
29     1
750    1
Name: target, dtype: int64

In [349]:
punct = '$,+\-/'
df['target'] = df['target'].str.replace('[a-z]', '').str.replace('[{}]'.format(punct), '').str.strip()

In [350]:
df['target'] = df['target'].apply(lambda x: None if x is None else int(x.split()[0]))

In [351]:
df.loc[330468]['target']

1215.0

In [352]:
df['target'].value_counts(dropna=False)

NaN         2481
225000.0    1806
275000.0    1650
250000.0    1644
350000.0    1641
            ... 
9100.0         1
515530.0       1
195050.0       1
780226.0       1
374521.0       1
Name: target, Length: 34233, dtype: int64

future engeneer

In [353]:
df.sample(5)

,status,propertyType,street,baths,homeFacts,city,schools,sqft,zipcode,beds,state,stories,target,stories_dop,street_v,drive_v,avenue_v,road_v,ridge_v,way_v,boulevard_v,parkway_v,circle_v,highway_v,lane_v,landing_v,trail_v,terrace_v,court_v,creek_v,place_v,roads_v,bend_v,falls_v,cliff_v,mountain_v,crest_v,hill_v,mill_v,point_v,loop_v,manor_v,heights_v,oaks_v,park_v,street_length,baths_new
18358,for sale,single family,oaklawn,None,"{'atAGlanceFacts': [{'factValue': '1957', 'fac...",winston salem,"[{'rating': ['7/10', '7/10', '6/10'], 'data': ...",1132.0,27107.0,3.0,NC,1.0,157500.0,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,NaN
275193,for sale,lot land,None,None,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",olympia,"[{'rating': ['5/10', '8/10', '9/10'], 'data': ...",NaN,98512.0,NaN,WA,NaN,120000.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,NaN
66876,active,single family,39th,2,"{'atAGlanceFacts': [{'factValue': '1970', 'fac...",ocala,"[{'rating': ['5', '4', '5', 'NR', 'NR', '1', '...",1732.0,34470.0,3.0,FL,1,179900.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2.0
105514,for sale,single family,manning,2,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",indianapolis,"[{'rating': ['5/10'], 'data': {'Distance': ['2...",1786.0,46228.0,3.0,IN,NaN,159900.0,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2.0
151410,for sale,single family,12th,1,"{'atAGlanceFacts': [{'factValue': '1925', 'fac...",san antonio,"[{'rating': ['9/10', '7/10', '9/10'], 'data': ...",590.0,98119.0,1.0,WA,1.0,1350000.0,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1.0
